In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

# settings
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['text.usetex']=False
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
plt.rcParams['savefig.dpi']=600
sns.set_style("ticks")

In [6]:
seq = torch.rand((1, 10, 1))

In [8]:
nn.LSTM(1, 16, 1)(seq)

(tensor([[[ 0.0837, -0.0277,  0.0181,  0.0360,  0.1127, -0.0324,  0.0900,
            0.0611, -0.0694, -0.0026, -0.0110,  0.0574,  0.0903,  0.0718,
            0.0632, -0.0870],
          [ 0.0838, -0.0287,  0.0198,  0.0352,  0.1134, -0.0333,  0.0888,
            0.0615, -0.0697, -0.0016, -0.0129,  0.0584,  0.0899,  0.0730,
            0.0616, -0.0878],
          [ 0.0837, -0.0274,  0.0175,  0.0362,  0.1125, -0.0320,  0.0904,
            0.0610, -0.0693, -0.0029, -0.0104,  0.0570,  0.0905,  0.0713,
            0.0638, -0.0867],
          [ 0.0836, -0.0264,  0.0159,  0.0369,  0.1117, -0.0311,  0.0916,
            0.0606, -0.0690, -0.0039, -0.0086,  0.0560,  0.0909,  0.0701,
            0.0653, -0.0858],
          [ 0.0840, -0.0321,  0.0255,  0.0326,  0.1157, -0.0365,  0.0846,
            0.0629, -0.0707,  0.0019, -0.0191,  0.0617,  0.0885,  0.0772,
            0.0561, -0.0906],
          [ 0.0828, -0.0172,  0.0015,  0.0425,  0.1043, -0.0224,  0.1015,
            0.0569, -0.0661, -0.0123

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.LN = nn.LayerNorm(1)
        self.lstm1 = nn.LSTM(1, 16, 1)
        self.fc1 = nn.Linear(1, 1)

    def forward(self, x):
        x = nn.LN(x)
        x, (h, cn) = self.lstm1(x)
        return self.fc1(x)

    def get_embedding(self, x):
        return self.forward(x)

In [ ]:
class SiameseNet(nn.Module):
    def __init__(self, embedding_net):
        super(SiameseNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        return output1, output2

    def get_embedding(self, x):
        return self.embedding_net(x)


class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super().__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        output3 = self.embedding_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)